<a href="https://colab.research.google.com/github/jivemachine/DCGAN/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

In [2]:
# pulling fashion mnist dataset from keras' dataset library
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255 # normalizing train data
X_test = X_test.astype(np.float32) / 255 # normalizing test data
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
coding_size = 100
# building the generator
generator = keras.models.Sequential([
  keras.layers.Dense(7 * 7 * 128, input_shape=[coding_size]),
  keras.layers.Reshape([7, 7, 128]),
  keras.layers.BatchNormalization(),
  keras.layers.Conv2DTranspose(1, kernel_size=5, strides=2, padding='same', activation='selu'),
  keras.layers.BatchNormalization(),
  keras.layers.Conv2DTranspose(1, kernel_size=5, strides=2, padding='same', activation='tanh')              
])


In [6]:
# building discriminator
discriminator = keras.models.Sequential([
  keras.layers.Conv2D(64, kernel_size=5, strides=2, padding='same', activation=keras.layers.LeakyReLU(0.2), input_shape=[28, 28, 1]),
  keras.layers.Dropout(0.4),
  keras.layers.Conv2D(128, kernel_size=5, strides=2, padding='same', activation=keras.layers.LeakyReLU(0.2)),
  keras.layers.Dropout(0.4),
  keras.layers.Flatten(),
  keras.layers.Dense(1, activation='sigmoid')                                                                                    
])

In [7]:
gan = keras.models.Sequential([generator, discriminator])

In [8]:
# compiling the discriminator & the gan
discriminator.compile(loss='binary_crossentropy', optimizer="rmsprop")
discriminator.trainable=False
gan.compile(loss='binary_crossentropy', optimizer='rmsprop')

In [9]:
# we need to reshape the training data to the same range as the generator
X_train = X_train.reshape(-1, 28, 28, 1) * 2. - 1. # reshape & rescale